In [41]:
import numpy as np
from lookup_table import CaseNum2EdgeOffset, getCaseNum
import trimesh
import os
import time

In [42]:
def marching_cube(thres,cells):
    # vertices use dictionary to avoid duplicate axes
    vertex_array = {}
    face_array = []
    t1 = time.time()
    # -------------------TODO------------------ 
    # compute vertices and faces
    # vertices: [N, 3]
    # faces: [M, 3], e.g. np.array([[0,1,2]]) means a triangle composed of vertices[0], vertices[1] and vertices[2]
    # for-loop is allowed to reduce difficulty
    # -------------------TODO------------------ 
    for x in range(cells.shape[0]-1):
        for y in range(cells.shape[1]-1):
            for z in range(cells.shape[2]-1):
                # cell = cells[x:x+2,y:y+2,z:z+2]
                casenum = getCaseNum(x,y,z, thres,cells)
                vertex = []
                for node in casenum:
                    if node == -1:
                        continue
                    vec0=(x+CaseNum2EdgeOffset[node][0],y+CaseNum2EdgeOffset[node][1],z+CaseNum2EdgeOffset[node][2])
                    vec1=(x+CaseNum2EdgeOffset[node][3],y+CaseNum2EdgeOffset[node][4],z+CaseNum2EdgeOffset[node][5])
                    tx,ty,tz=np.array(vec0)-cells[vec0]/(cells[vec1]-cells[vec0])*(np.array(vec1)-np.array(vec0))
                    tmp=(tx,ty,tz)
                    if vertex_array.get(tmp) is None:
                        vertex_array[tmp]=len(vertex_array)
                    vertex.append(vertex_array[tmp])
                    if(len(vertex)==3):
                        face_array.append(vertex)
                        vertex=[]
    l=np.zeros((len(vertex_array),3))
    for k,v in vertex_array.items():
        l[v]=k
    vertex_array=l
    t2 = time.time()
    print("\nTime taken by algorithm\n"+'-'*40+"\n{} s".format(t2-t1))
    return np.array(vertex_array), np.array(face_array)

In [43]:
# reconstruct these two animals
shape_name_lst = ['spot', 'bob']
for shape_name in shape_name_lst:
    data = np.load(os.path.join('data', shape_name + '_cell.npy'))
    verts, faces = marching_cube(0, data)
    mesh = trimesh.Trimesh(vertices=verts, faces=faces)
    mesh_txt = trimesh.exchange.obj.export_obj(mesh)
    with open(os.path.join('../results', shape_name + '.obj'),"w") as fp:
        fp.write(mesh_txt)


Time taken by algorithm
----------------------------------------
5.204808473587036 s

Time taken by algorithm
----------------------------------------
5.659600019454956 s
